# Pairs Trading Example
### Pairs trading essentially uses statistical arbitrage and mean reversion to bet on a pair of financial instruments reverting back to a correlated pattern. 
### If a pair is correlated and their spread widens enough, we can short the overpriced security and long the underpriced security. Then when the spread reverts back to the mean, we cash out. 
### To find similar pairs of stocks, use cointegration (for some reason more accurate than correlation in the long term)


In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

import datetime
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="whitegrid")

pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as pdr
yf.pdr_override()

In [ ]:
# Get historical stock data from 2016-2020 for ETFs

start = datetime.datetime(2016, 1, 1)
end = datetime.datetime(2020, 1, 1)

stockz=sorted(["EEM", "SPY", "GDX", "XLF", "XOP", "AMLP", "FXI", "QQQ", "EWZ", "EFA", "USO", "HYG", "IAU", "IWM", "XLE", "XLU", "IEMG", "GDXJ", "SLV", "VWO", "XLP", "XLI", "OIH", "LQD", "XLK", "VEA", "TLT", "IEFA", "XLV", "EWJ", "GLD", "IYR", "BKLN", "EWH", "ASHR", "XLB", "RSX", "JNK", "KRE", "XBI", "AGG", "VNQ", "GOVT", "UNG", "IVV", "XLY", "EWT", "PFF", "XLRE", "MCHI", "INDA", "BND", "USMV", "EZU", "SMH", "XRT", "EWY", "IEF", "SPLV", "XLC", "IJR", "VIXY", "EWG", "EWW", "VTI", "VGK", "IBB", "PGX", "VOO", "EMB", "SCHF", "VEU", "SJNK", "EMLC", "XME", "DIA", "EWA", "VCSH", "JPST", "MLPA", "VCIT", "ITB", "ACWI", "KWEB", "EWC", "EWU", "BNDX", "SHY", "VT", "IWD", "VXUS", "MBB", "ACWX", "XHB", "BSV", "SHV", "FEZ", "IWF", "IGSB", "SPYV", "ITOT", "FPE", "FVD", "SHYG", "VYM", "BBJP", "DGRO", "KBE", "VTV", "SPAB", "SPIB", "IWR", "DBC", "BIL", "SPSB", "FLOT", "GLDM", "VIG", "XES", "SCHE", "TIP", "PDBC", "SPYG", "MINT", "SCZ", "SPDW", "PCY", "USHY", "IXUS", "NEAR", "EPI", "SPLG", "HYLB", "AAXJ", "SPEM", "VMBS", "BIV", "QUAL", "ILF", "EWP"])

df = pdr.get_data_yahoo(stockz, start, end)['Close']
df.tail()

In [ ]:
#Check for and remove null values
for col in df.columns:
    if df[col].isnull().any():
        df.drop(col, axis = 1, inplace=True)

# for col in df.columns:
#     print(col, (df[col].isnull().sum()))

print(len(df.columns))

In [ ]:
def find_cointegrated_pairs(data, threshold = 0.05):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < threshold:
                pairs.append((keys[i], keys[j], pvalue))
    return score_matrix, pvalue_matrix, pairs

In [ ]:
#scores, pvalues, pairs = find_cointegrated_pairs(df)

#RUN HIDDEN CODE BELOW TO SAVE TIME

In [ ]:
# import seaborn
# fig, ax = plt.subplots(figsize=(100,100))
# seaborn.heatmap(pvalues, xticklabels=stockz, yticklabels=stockz, cmap='RdYlGn_r' , mask = (pvalues >= 0.01))
# fig = ax.get_figure()
# #fig.savefig('heatmap.png')
# print(len(pairs), pairs)

In [ ]:
def total_pairs_under_threshold(threshold):
    totalsum = 0
    for i in range(len(pvalues)):
        cursum = sum(pvalues[i] <= threshold)
        totalsum += cursum
        print(stockz[i], cursum)
    return totalsum

def pairs_under_threshold(pairs, threshold):
    new_pairs = []
    for pair in pairs:
        if pair[2] <= threshold:
            new_pairs.append(pair)
    return new_pairs

new_pairs = pairs_under_threshold(pairs, 0.01)
new_pairs

In [ ]:
def get_weights(pairs, max_allocation = .01):
    weights = []
    for p in pairs:
        # print(p)
        weights.append(1 / p[2])

    weights = np.array(weights)
    total = np.sum(weights)

    normalized = weights / total

    while any((normalized) > max_allocation):
        #print(normalized.max())
        normalized[normalized.argmax()] = max_allocation
        normalized = normalized / sum(normalized)

    scale_factor = 1 / normalized.min()
    scaled = scale_factor * normalized
    return scaled

get_weights(pairs)

In [ ]:
# Calculating Spread

def spread_difference(s1, s2):
    S1 = df[s1]
    S2 = df[s2]

    S1 = sm.add_constant(S1)
    results = sm.OLS(S2, S1).fit()
    S1 = S1[s1]
    b = results.params[s1]

    spread = S2 - b * S1
    spread.plot(figsize=(12,6))
    plt.axhline(spread.mean(), color='black')
    plt.xlim('2016-01-01', '2020-01-01')
    plt.legend(['Spread']);

spread_difference('XLRE', 'XLU')

In [ ]:
# Calculating Spread Ratio

def spread_ratio(s1, s2):
    S1 = df[s1]
    S2 = df[s2]
    ratio = S1/S2
    ratio.plot(figsize=(12,6))
    plt.axhline(ratio.mean(), color='black')
    plt.xlim('2016-01-01', '2020-01-01')
    plt.legend(['Price Ratio']);
    return ratio

ratio = spread_ratio('XLRE', 'XLU')
ratio

In [ ]:
# Calculate z score
def zscore(series):
    return (series - series.mean()) / np.std(series)


zscore(ratio).plot(figsize=(12,6))
plt.axhline(zscore(ratio).mean())
plt.axhline(1.0, color='red')
plt.axhline(-1.0, color='green')
plt.xlim('2016-01-01', '2020-01-01')
plt.show()

### Modeling

In [ ]:
# Create train and test data

ratios = df['XLRE'] / df['XLU'] 
train_test_index = int(len(ratios) * .70)
train = ratios[:train_test_index]
test = ratios[train_test_index:]

In [ ]:
# Visualizing ratio and moving average parameters

ratios_mavg5 = train.rolling(window=5, center=False).mean()
ratios_mavg60 = train.rolling(window=60, center=False).mean()
std_60 = train.rolling(window=60, center=False).std()
zscore_60_5 = (ratios_mavg5 - ratios_mavg60)/std_60
plt.figure(figsize=(12, 6))
plt.plot(train.index, train.values)
plt.plot(ratios_mavg5.index, ratios_mavg5.values)
plt.plot(ratios_mavg60.index, ratios_mavg60.values)
plt.legend(['Ratio', '5d Ratio MA', '60d Ratio MA'])

plt.ylabel('Ratio')
plt.show()

In [ ]:
# Visualizing z score in comparison to mean: we can see that with a z-score over 1 it tends to revert back to the mean of 0, and same for a z-score under -1
plt.figure(figsize=(12,6))
zscore_60_5.plot()
plt.xlim('2016-03-25', '2018-07-01')
plt.axhline(0, color='black')
plt.axhline(1.0, color='red', linestyle='--')
plt.axhline(-1.0, color='green', linestyle='--')
plt.legend(['Rolling Ratio z-Score', 'Mean', '+1', '-1'])
plt.show()

In [ ]:
# Visualizing buy/sell signals in terms of spread ratio

plt.figure(figsize=(12,6))

train[160:].plot()
buy = train.copy()
sell = train.copy()
buy[zscore_60_5>-1] = 0
sell[zscore_60_5<1] = 0
buy[160:].plot(color='g', linestyle='None', marker='^')
sell[160:].plot(color='r', linestyle='None', marker='^')
x1, x2, y1, y2 = plt.axis()
plt.axis((x1, x2, ratios.min(), ratios.max()))
plt.xlim('2016-08-15','2018-07-07')
plt.legend(['Ratio', 'Buy Signal', 'Sell Signal'])
plt.show()

In [ ]:
# Visualizing buy/sell signals in terms of security prices

plt.figure(figsize=(12,7))
S1 = df['XLRE'].iloc[:train_test_index]
S2 = df['XLU'].iloc[:train_test_index]

S1[60:].plot(color='b')
S2[60:].plot(color='c')
buyR = 0*S1.copy()
sellR = 0*S1.copy()

# When you buy the ratio, you buy stock S1 and sell S2
buyR[buy!=0] = S1[buy!=0]
sellR[buy!=0] = S2[buy!=0]

# When you sell the ratio, you sell stock S1 and buy S2
buyR[sell!=0] = S2[sell!=0]
sellR[sell!=0] = S1[sell!=0]

buyR[60:].plot(color='g', linestyle='None', marker='^')
sellR[60:].plot(color='r', linestyle='None', marker='^')
x1, x2, y1, y2 = plt.axis()
plt.axis((x1, x2, min(S1.min(), S2.min()), max(S1.max(), S2.max())))
plt.ylim(25, 60)
plt.xlim('2016-03-22', '2018-07-04')

plt.legend(['XLRE', 'XLU', 'Buy Signal', 'Sell Signal'])
plt.show()

In [ ]:
# Trade with moving average strategy
def single_trade(s1, s2, window1, window2, w = 1, use_ratio = False):
    
    # If window length is 0, algorithm doesn't make sense, so exit
    if (window1 == 0) or (window2 == 0):
        return 0
    
    S1 = df[s1].iloc[train_test_index:]
    S2 = df[s2].iloc[train_test_index:]
    # Compute rolling mean and rolling standard deviation
    ratios = S1/S2
    ma1 = ratios.rolling(window=window1,
                               center=False).mean()
    ma2 = ratios.rolling(window=window2,
                               center=False).mean()
    std = ratios.rolling(window=window2,
                        center=False).std()
    zscore = (ma1 - ma2)/std
    
    # Simulate trading
    # Start with no money and no positions
    money = 0
    countS1 = 0
    countS2 = 0
    for i in range(len(ratios)):
        # Sell short if the z-score is > 1
        if zscore[i] < -1:
            if use_ratio:
                countS1 -= 1 * ratios[i] * w
                money += (S1[i] - S2[i]) * w * ratios[i]
            else:
                money += (S1[i] - S2[i]  * ratios[i]) * w
                countS1 -= 1 * w
            
            countS2 += 1 * ratios[i] * w
            #print('Selling Ratio %s %s %s %s'%(money, ratios[i], countS1,countS2))
        # Buy long if the z-score is < -1
        elif zscore[i] > 1:
            if use_ratio:
                money -= (S1[i] - S2[i]) * w * ratios[i]
                countS1 += 1 * ratios[i] * w
            else:
                money -= (S1[i] - S2[i] * ratios[i]) * w 
                countS1 += 1 * w
            
            countS2 -= 1 * ratios[i] * w

        # Clear positions if the z-score between -.5 and .5
        elif abs(zscore[i]) < 0.75:
            money += S1[i] * countS1 + S2[i] * countS2
            countS1 = 0
            countS2 = 0
            #print('Exit pos %s %s %s %s'%(money,ratios[i], countS1,countS2))
            
            
    return money

In [ ]:
# Trades all pairs of ETFs
def multiple_trades(pairs, window1, window2, weights, use_ratio = False, use_weight = False):
    total_money = 0
    negative_returns = 0
    for i in range(len(pairs)):
        pair = pairs[i]
        if use_weight:
            cur_weight = weights[i]
        else:
            cur_weight = 1
        S1 = pair[0]
        S2 = pair[1]
        pair_money = single_trade(S1, S2, window1, window2, cur_weight, use_ratio)
        total_money += pair_money
        if pair_money < 0:
            negative_returns += 1
    
    #print("Fraction of pairs with negative returns: " + str(negative_returns) + "/" + str(len(pairs)))

    if use_weight:
        scale = sum(weights) / len(weights)
    else:
        scale = 1
    #print("Scale: ", scale)
    #print("Ratio for S1 Used: ", use_ratio)
    avg_money = (total_money / (len(pairs) * scale))

    #print("Average money made per pair scaled per unit : " + str(avg_money))
    return avg_money, negative_returns, len(pairs), scale

### Results

In [ ]:
single_trade('XLRE', 'XLU', 60, 5, 1)

In [ ]:
# Pairs with p value < 0.05


multiple_trades(pairs, 60, 5, get_weights(pairs, .01), use_ratio=True, use_weight=False)

In [ ]:
# Pairs with p value < 0.01
pairs_1 = pairs_under_threshold(pairs, 0.01)

multiple_trades(pairs_1, 60, 5, get_weights(pairs_1, .01), use_ratio=True, use_weight=True)

### Optimization

In [ ]:
window_1_vals = [30, 60, 90]
window_2_vals = [3, 5, 10]
allocations = [.01, .05, .1]
p_vals = [.05, .01, .005]
use_r = [True, False]
use_w = [True, False]

In [ ]:
parameters = []
results = []
count = 1
for w1 in window_1_vals:
    for w2 in window_2_vals:
        for a in allocations:
            for p in p_vals:
                cur_pairs = pairs_under_threshold(pairs, p)
                if (p == .005):
                    a = max(a, .025)
                cur_w = get_weights(cur_pairs, a)
                for r in use_r:
                    for w in use_w:
                        parameters.append((w1, w2, a, p, r, w))
                        results.append(multiple_trades(cur_pairs, w1, w2, cur_w, use_ratio=r, use_weight=w))
                        print(count, (w1, w2, a, p, r, w))
                        count += 1

In [ ]:
def get_results(p, pretty_print = False):
    if p in parameters:
        i = parameters.index(p)
        cur_result = results[i]
 
    else:
        cur_pairs = pairs_under_threshold(pairs, p[3])
        cur_w = get_weights(cur_pairs, p[2])
        cur_result = multiple_trades(cur_pairs, p[0], p[1], cur_w, use_ratio=p[4], use_weight=p[5])
    
    if pretty_print:
        print("Window 1 Length: ", p[0])
        print("Window 2 Length: ", p[1])
        print("Max Weightage to Pair: ", p[2])
        print("P-Value Threshold for Pair: ", p[3])
        print("Use Ratios to Scale Stock 1: ", p[4])
        print("Use Weights to Scale Stocks: ", p[5])
        print()
        print("Average money made per pair scaled per unit: ", cur_result[0])
        print("Fraction of pairs with negative returns: ", str(cur_result[1]), "/", str(cur_result[2]))
        print("Scale Factor: ", cur_result[3])

    return cur_result[0], cur_result[1] / cur_result[2]

In [ ]:
get_results((60, 5, .01, .01, False, True), pretty_print=True)

In [ ]:
optimized = sorted(parameters, key = lambda p: get_results(p)[0], reverse=True)
safest = sorted(parameters, key = lambda p: get_results(p)[1])

optimized

In [ ]:
#Best Overall
get_results(optimized[0], pretty_print=True)

In [ ]:
#Best Using Weights
get_results((60, 3, 0.01, 0.01, True, True), pretty_print=True)

In [ ]:
#Most Consistent
get_results((60, 3, 0.01, 0.01, False, False), pretty_print=True)